# Hierarchical RNN for dialogue
The baseline hierarchical model from "Hierarchical Text Generation and Planning for Strategic Dialogue."

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [18]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import SentenceParser

In [19]:
train_iterations = 300
learning_rate = 0.1
max_input_length = 20
max_output_length = 20
unk_threshold = 20

### Sentence-level parsing
Not dealing with the final action data for now. Training examples are lists of utterances.

In [20]:
parser = SentenceParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/tmp/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [21]:
parser.parse()

In [194]:
parser.vocab.append('<eos>')

### Hierarchical Agent

In [341]:
class HierarchicalAgent(Agent):
    def __init__(self, max_turns=25, **kwargs):
        self.max_turns = max_turns # Max number of dialogue turns (i.e., utterances)
        super(HierarchicalAgent, self).__init__(**kwargs)
    
    def _init_placeholders(self):
        self.encoder_inputs = tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="encoder_inputs")

        self.encoder_lengths = tf.placeholder(
            shape=[None, None],
            dtype=tf.int32,
            name="encoder_lengths")

        self.decoder_inputs= tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="decoder_inputs")

        self.decoder_targets = tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="decoder_targets")

        self.decoder_lengths = tf.placeholder(
            shape=[None, None],
            dtype=tf.int32,
            name="decoder_lengths")
        
    def encoding_layer(self):
        self.encoder_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.LSTMCell(
                self.hidden_dim, 
                initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=1),
                reuse=tf.AUTO_REUSE)
            for _ in range(self.num_layers)])
        
        self.encoder_final_states = tf.map_fn(
            fn=self.encode_step, 
            elems=(self.embedded_encoder_inputs, self.encoder_lengths), dtype=tf.float32)
        

    def encode_step(self, args):
        embedded_encoder_inputs = args[0]
        encoder_lengths = args[1]
        
        encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
            cell=self.encoder_cell,
            inputs=embedded_encoder_inputs,
            sequence_length=encoder_lengths,
            dtype=tf.float32)
        
        return encoder_final_state[-1][1]
        
    def context_layer(self):
        self.context_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.LSTMCell(
                self.hidden_dim, 
                initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            for _ in range(self.num_layers)])
        
        context_outputs, context_final_state = tf.nn.dynamic_rnn(
            cell=self.context_cell,
            inputs=self.encoder_final_states,
            sequence_length=[self.max_turns] * self.batch_size, # TODO: check this
            dtype=tf.float32,
            scope="context_layer")
        
        self.context_outputs = context_outputs
        
    def decoding_layer(self):
        self.decoder_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.GRUCell(self.hidden_dim) for _ in range(self.num_layers)])
        
        self.output_layer = Dense(
            units=self.vocab_size,
            kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
        
        self.decoding_training()
#         self.decoding_inference()
    
    def decoding_training(self):
        self.training_logits = tf.map_fn(
            fn=self.decoding_training_step,
            elems=(self.context_outputs, self.embedded_decoder_inputs, self.decoder_lengths),
            dtype=tf.float32)
        
    def decoding_training_step(self, args):
        context_state = args[0]
        embedded_decoder_inputs = args[1]
        decoder_lengths = args[2]
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
            inputs=embedded_decoder_inputs,
            sequence_length=decoder_lengths,
            time_major=False)
        
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            helper=training_helper,
            initial_state=(context_state,context_state),
            output_layer=self.output_layer)

        training_outputs = tf.contrib.seq2seq.dynamic_decode(
            decoder=training_decoder,
            impute_finished=True,
            maximum_iterations=self.max_output_length)[0]
        
        return training_outputs.rnn_output # logits
    
    def build_graph(self):
        self._init_placeholders()
        self._define_embedding()
        self.encoding_layer()
        self.context_layer()
        self.decoding_layer()
        
    def train_dict(self, X, y):
        decoder_inputs = [["<START> " + sent for sent in dialogue] for dialogue in y]
        decoder_targets = [[sent + " <END>" for sent in dialogue] for dialogue in y]
        
        encoder_inputs, encoder_lengths = self.prepare_data(X)
        decoder_inputs, _ = self.prepare_data(y)
        decoder_targets, decoder_lengths = self.prepare_data(decoder_targets)

        return {self.encoder_inputs: encoder_inputs,
            self.decoder_inputs: decoder_inputs,
            self.decoder_targets: decoder_targets,
            self.encoder_lengths: encoder_lengths,
            self.decoder_lengths: decoder_lengths}
    
    def prepare_data(self, data):
        batch_size = len(data)
        max_length = self.max_input_length
        
        index = dict(zip(self.vocab, range(len(self.vocab))))
        unk_index = index['$UNK']
        
        new_data = np.zeros((self.max_turns, batch_size, max_length), dtype='int')
        ex_lengths = np.zeros((self.max_turns, batch_size))
        max_num_turns = 0
        for batch in range(batch_size):
            num_turns = len(data[batch])
            if num_turns > max_num_turns:
                max_num_turns = num_turns
            for turn in range(min(num_turns, self.max_turns)):
                ex_len = min([len(data[batch][turn]), self.max_input_length])
                ex_lengths[turn][batch] = ex_len
                vals = data[batch][turn][-max_length: ].split()
                vals = [index.get(w, unk_index) for w in vals]
                temp = np.zeros((max_length,), dtype='int')
                temp[0: len(vals)] = vals
                new_data[turn][batch] = temp
        return new_data, ex_lengths
    
    def get_cost_function(self, **kwargs):
        # Need to flatten the logits (TODO: check this):
        training_logits = tf.reshape(self.training_logits, [self.batch_size, -1, self.vocab_size])
        decoder_targets = tf.reshape(self.decoder_targets, [self.batch_size, -1])
        
        decoder_lengths = tf.reduce_sum(self.decoder_lengths, 0)
        masks = tf.sequence_mask(decoder_lengths, self.max_output_length, dtype=tf.float32, name='masks')
        cost = tf.contrib.seq2seq.sequence_loss(
            logits=training_logits,
            targets=decoder_targets,
            weights=masks)
        return cost

In [342]:
tf.reset_default_graph()
a = HierarchicalAgent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

In [343]:
a.build_graph()

Tensor("Shape:0", shape=(3,), dtype=int32)


#### Data parsing

In [344]:
train_data = []
with open("../data/tmp/train.txt", "r") as train_file:
    for line in train_file:
        train_example = json.loads(line)
        train_data.append((train_example[:-1], train_example))

In [345]:
X, y = zip(*train_data)

In [346]:
a.fit(X, y)

Tensor("Shape:0", shape=(3,), dtype=int32)


InvalidArgumentError: assertion failed: [Expected shape for Tensor context_layer/sequence_length:0 is ] [25] [ but saw shape: ] [1028]
	 [[Node: context_layer/Assert/Assert = Assert[T=[DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](context_layer/All, context_layer/Assert/Assert/data_0, context_layer/stack, context_layer/Assert/Assert/data_2, context_layer/Shape_1)]]

Caused by op 'context_layer/Assert/Assert', defined at:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/

## External RNN code
Code from "Hierarchical Recurrent Encoder-Decoder for Generative Context-Aware Query Suggestion"
by Sordoni et al. (2015)

Paper: https://arxiv.org/abs/1507.02221
Repository: https://github.com/tscheepers/hred-attention-tensorflow

In [9]:
sys.path.append("../src/models/external/")
import layers

In [10]:
class HierarchicalAgent(Agent):    
    def encoding_layer(self):
        self.eoq_mask = tf.expand_dims(tf.cast(tf.not_equal(self.encoder_inputs, self.vocab.index("<eos>")), tf.float32), 2)
        
        self.sentence_encoder = tf.scan(
            lambda result_prev, x: layers.gru_layer_with_reset(
                result_prev[1],
                x,
                name='sentence_encoder',
                x_dim=self.embed_dim,
                y_dim=self.hidden_dim,
                reuse=tf.AUTO_REUSE
            ),
            (self.embedded_encoder_inputs, self.eoq_mask),
            initializer=tf.zeros((2, self.batch_size, self.hidden_dim))
        )
        
        self.encoder_final_states, _ = tf.unstack(self.sentence_encoder, axis=1)
        
    def context_layer(self):
        context_encoder = tf.scan(
            lambda result_prev, x: layers.gru_layer_with_retain(
                result_prev[1],
                x,
                name='context_encoder',
                x_dim=self.hidden_dim,
                y_dim=self.context_dim,
                reuse=tf.AUTO_REUSE
            ),
            (self.encoder_final_states, self.eoq_mask),
            initializer=tf.zeros((2, self.batch_size, self.context_dim))
        )
        
        self.context_final_states, _ = tf.unstack(context_encoder, axis=1)

    def decoding_layer(self):
        num_of_steps = tf.shape(self.encoder_inputs)[0]
        
        decoder = tf.scan(
            lambda result_prev, x: layers.gru_layer_with_state_reset(
                result_prev,
                x,
                name='decoder',
                x_dim=self.embed_dim,
                h_dim=self.context_dim,
                y_dim=self.hidden_dim,
                reuse=tf.AUTO_REUSE
            ),
            (self.embedded_encoder_inputs, self.eoq_mask, self.context_final_states),
            initializer=tf.zeros((self.batch_size, self.hidden_dim))
        )
        
        flatten_decoder = tf.reshape(decoder, (-1, self.hidden_dim))
        flatten_embedder = tf.reshape(self.embedded_encoder_inputs, (-1, self.embed_dim))
        
        output_layer = layers.output_layer(
            flatten_embedder,
            flatten_decoder,
            x_dim=self.embed_dim,
            h_dim=self.hidden_dim,
            y_dim=self.vocab_size,
            reuse=tf.AUTO_REUSE
        )
        
        flatten_logits = layers.logits_layer(
            output_layer,
            x_dim=self.vocab_size,
            y_dim=self.vocab_size,
            reuse=tf.AUTO_REUSE
        )
        
        self.training_logits = tf.reshape(flatten_logits, (num_of_steps, self.batch_size, self.vocab_size))
    
    def build_graph(self):
        self.context_dim = 101
        
        self._init_placeholders()
        self._define_embedding()
        self.encoding_layer()
        self.context_layer()
        self.decoding_layer()

In [11]:
b = HierarchicalAgent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

In [12]:
# b.build_graph()

### Model training

In [13]:
train_data = []
with open("../data/processed/train.txt", "r") as train_file:
    for line in train_file:
        train_example = json.loads(line)
        train_data.append((
            train_example["input"],
            train_example["output"][0].split()))

In [16]:
X, y = zip(*train_data)
X = y

In [17]:
b.fit(X, y, save_path="../models/tmp")

ValueError: setting an array element with a sequence.